# London Cocktail Week 2024

London Cocktail Week is the world's original Cocktail Week and takes place in the cocktail capital of the world.

Every year, in October, London's best bars specially create Signature Cocktails, and allows cocktail enthusiasts to navigate their way through london, drink by drink.

More information can be found here: https://londoncocktailweek.com/


## What is this repository about?

As a cocktail enthusiast myself, I've attended London Cocktail Week since moving to London, and each year I scrape the website to allow me find where I want to visit in an easier way! With each year that passes, this notebook gets more advanced!

## The Code

### Import Statements

In [9]:
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import requests

### Parse the Website

In [10]:
# Get the PDF from the website
request = requests.get(
    'https://londoncocktailweek.com/bars/print/?collectionId=0&whatId=0&areaId=0&spiritId=0&openNow=0&search='
)

# Parse the website using Beautiful Soup
parsed_website = BeautifulSoup(request.text, 'html.parser')

### Collate bars into a Data Frame

In [11]:
# Create the dataframe
df = pd.DataFrame(columns=[
    'Bar Name',
    'Address',
    'Phone Number',
    'Description',
    'MON',
    'TUE',
    'WED',
    'THU',
    'FRI',
    'SAT',
    'SUN'
])

# Get the list of bars
bars = parsed_website.find('ul').findChildren("li", recursive=False)

# Loop through each bar
for i, bar in enumerate(bars):
    
    # Store the bar name, address, phone number & description
    bar_name     = bar.find('h2', {'class': 'bar_name'}).getText()
    address      = bar.find('div', {'class': 'text'}).getText()
    phone_number = bar.find('div', {'class': 'text--padded'}).getText()
    description  = bar.find('p', {'class': 'text text--padded'}).getText()

    # Get the opening hours container for each bar
    weekly_opening_hours_container = bar.find('ul', {'class': 'opening_hours__container'}).find_all('li', {'class': 'opening_hours__times'})

    # Store the times for each day of the week
    times = [opening_hours.find('li', {'class': 'text'}).getText() for opening_hours in weekly_opening_hours_container]

    # Save all bar data to the dataframe
    df.loc[i] = [
        bar_name,
        address,
        phone_number,
        description
    ] + times

In [12]:
csv_string = 'bar_csvs/bar_list_' + datetime.now().strftime("%Y_%m_%d_%H-%M-%S") + '.csv'
df.to_csv(csv_string)